In [5]:
from allensdk.brain_observatory.behavior.behavior_project_cache import BehaviorProjectCache as bpc
import os
import pandas as pd
from visual_behavior.data_access import loading

# get behavior session table
cache_dir = "//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/2020_cache/production_cache_temp"
manifest_path = os.path.join(cache_dir, 'manifest_new.json')
cache = bpc.from_lims(manifest_path)
behavior_session_table = cache.get_behavior_session_table()

# get filtered ophys experiment table
filtered_ophys_experiment_table = loading.get_filtered_ophys_experiment_table()

# inner merge two tables to get production behavior sessions
behavior_ophys_sessions = behavior_session_table.reset_index().merge(
    filtered_ophys_experiment_table[['ophys_session_id']],
    left_on='ophys_session_id',
    right_on='ophys_session_id',
    how='inner'
)
# cast OSID to int
behavior_ophys_sessions['ophys_session_id'] = behavior_ophys_sessions['ophys_session_id'].astype(int)

# get behavior sessions with missing `session_type`
behavior_ophys_sessions_missing_session_type = behavior_ophys_sessions[
    pd.isnull(behavior_ophys_sessions['session_type'])
]


# print to console
behavior_ophys_sessions_missing_session_type[[
    'ophys_session_id', 
    'behavior_session_id',
    'equipment_name',
    'date_of_acquisition',
    'session_type'
]]

Getting behavior-only session data. This might take a while...


,ophys_session_id,behavior_session_id,equipment_name,date_of_acquisition,session_type
233,1046147323,1046201300,CAM2P.3,2020-08-26 18:38:16,NaN
346,1047741404,1047774591,CAM2P.4,2020-09-01 19:44:37,NaN
531,1037942709,1037955916,CAM2P.3,2020-07-22 20:25:33,NaN
618,974019098,974140453,CAM2P.5,2019-10-30 15:40:18,NaN
999,1003207357,1003249011,CAM2P.3,2020-01-28 16:44:46,NaN


In [7]:
missing_osids

array([1046147323, 1047741404, 1037942709,  974019098, 1003207357])

In [9]:
missing_osids = behavior_ophys_sessions_missing_session_type['ophys_session_id'].values

et = cache.get_experiment_table()
et.query('ophys_session_id in @missing_osids')[[
    'ophys_session_id', 
    'behavior_session_id',
    'equipment_name',
    'date_of_acquisition',
    'session_type'
]]

Getting all ophys sessions. This might take a while.


,ophys_session_id,behavior_session_id,equipment_name,date_of_acquisition,session_type
ophys_experiment_id,,,,,
1046356415,1046147323,1046201300,CAM2P.3,2020-08-26 18:38:16,OPHYS_4_images_B
1047823920,1047741404,1047774591,CAM2P.4,2020-09-01 19:44:37,OPHYS_1_images_B
1038000628,1037942709,1037955916,CAM2P.3,2020-07-22 20:25:33,OPHYS_2_images_B_passive
974384470,974019098,974140453,CAM2P.5,2019-10-30 15:40:18,OPHYS_3_images_A
1003456278,1003207357,1003249011,CAM2P.3,2020-01-28 16:44:46,OPHYS_1_images_B
